###### ECE194N: Homework 3
###### Topics: RNN Problem
###### Due: May 29


-------------------------------------------------

RNN: In this problem we will be implementing an RNN. The objective of the RNN
is that given a sequence of words, it will try to guess the next word. For this task we
will follow the RNN tutorial for Tensorflow which can be found from this link:

https://www.tensorflow.org/tutorials/recurrent


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

import reader


C:\Users\Karma\Anaconda3\envs\tflo\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### (a.) Download the data and read the reader code from the tutorial. 

Write a small script that displays the word ids and the sentences for the first 10 samples in the validation set. 
##### Hint: It should be something like this: ”I like apples” [1,2,3] where 1:I, 2:like, 3:apples.

In [33]:
# Read PTB sample data 
data_path = "simple-examples/data/"
raw_data = reader.ptb_raw_data(data_path)
train_data, valid_data, test_data, _ = raw_data

# Read the words from the valid data file
data = reader._read_words(data_path+"ptb.valid.txt")

# Print 10 sample words
word_valid_sample = ""
for idx in valid_data[0:10]:
    print(idx,"\t:",data[idx])
    word_valid_sample = word_valid_sample + " " + data[idx]
print("\n")
print("\"",word_valid_sample,"\" ", valid_data[0:10])

1132 	: know
93 	: new
358 	: excesses
5 	: their
329 	: lines
51 	: consumer
9836 	: savings
6 	: telephones
326 	: while
2476 	: colombia


"  know new excesses their lines consumer savings telephones while colombia "  [1132, 93, 358, 5, 329, 51, 9836, 6, 326, 2476]


### (b) Design a model to classify these 10 classes. 
##### Choose parameters of your model architecture such as number of layers, number of neurons and the loss function. 
Comment on these choices. Implement this model in Tensorflow and train the model weights using the training set. Plot the training and testing loss and acc 

